# BMS with AutoRA State

The tutorial illustrates how to use the BMS Regressor with the AutoRA state on some synthetic data.

In [ ]:
import numpy as np
import pandas as pd
from autora.variable import VariableCollection, Variable
from autora.state.standard import StandardState

## Create a Variable Collection
First, we need to define our experimental space. Here, we consider two input variables, $x_1$, $x_2$, and one output variable, $y$.

In [ ]:
variables=VariableCollection(
        independent_variables=[Variable("x_1", allowed_values=np.linspace(-10, 10, 10)),
                               Variable("x_2", allowed_values=np.linspace(-10, 10, 10))],
        dependent_variables=[Variable("y")]
    )

## Defining the State

We can define an initial state for our discovery problem based on the variable specification above.


In [ ]:
state = StandardState(
    variables=variables
)

## Defining an Experiment Runner

 Next, we will define a synthetic experiment runner that takes input variables $x_1$, $x_2$, and produces output the variable $y$. The input the the experiment runner has to be called ``conditions`` so we can later integrate it into the autora cycle. 

In [ ]:
def synthetic_experiment_runner(conditions: pd.DataFrame, variables: VariableCollection):
    experiment_data = pd.DataFrame(conditions)
    y = conditions[variables.independent_variables[0].name]**2 \
        + conditions[variables.independent_variables[1].name]**2
    experiment_data[variables.dependent_variables[0].name] = y
    return experiment_data

Let's try out our experiment runner with a basic dataset.

In [ ]:
conditions = pd.DataFrame({"x_1": [1, 2, 3], "x_2": [1, 2, 3]})
experiment_data = synthetic_experiment_runner(conditions, variables)
experiment_data

,x_1,x_2,y
0,1,1,2
1,2,2,8
2,3,3,18


Finally, we can wrap the experiment runner into a state function.

In [ ]:
from autora.state import on_state
experiment_runner = on_state(function=synthetic_experiment_runner, output=["experiment_data"])

## Defining an Experimentalist

Next, we define a grid pool experimentalist and wrap it into a state function. The grid pool experimentalist expects the ``variables`` object as input. By wrapping it into a state function, it will automatically pull the ``variables`` object from the state.


In [ ]:
from autora.experimentalist.grid_ import grid_pool

experimentalist = on_state(function=grid_pool, output=["conditions"])

## Add Some Data to the State

We can let the grid experimentalist generate initial conditions and add them to the state.

In [ ]:
state = experimentalist(state)

We can directly observe the conditions added to the state.

In [ ]:
state.conditions

,x_1,x_2
0,-10.0,-10.000000
1,-10.0,-7.777778
2,-10.0,-5.555556
3,-10.0,-3.333333
4,-10.0,-1.111111
...,...,...
95,10.0,1.111111
96,10.0,3.333333
97,10.0,5.555556
98,10.0,7.777778


Now that we have some experimental conditions, we can generate experimental observations from the experiment runner.

In [ ]:
state = experiment_runner(state)
state.experiment_data

,x_1,x_2,y
0,-10.0,-10.000000,200.000000
1,-10.0,-7.777778,160.493827
2,-10.0,-5.555556,130.864198
3,-10.0,-3.333333,111.111111
4,-10.0,-1.111111,101.234568
...,...,...,...
95,10.0,1.111111,101.234568
96,10.0,3.333333,111.111111
97,10.0,5.555556,130.864198
98,10.0,7.777778,160.493827


## Define BMS Theorist

Next, we can define a BMS theorist and wrap it into a state function.

In [ ]:
from autora.state.wrapper import state_fn_from_estimator
from autora.theorist.bms import BMSRegressor

theorist = state_fn_from_estimator(BMSRegressor(epochs=500))

We can apply the BMS theorist to the state to obtain a model of the experimental data.

In [ ]:
state = theorist(state)

INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 500/500 [00:34<00:00, 14.46it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


Once fitted, we can print the model.

In [ ]:
state.model.__repr__()


'((x_2 * x_2) + 40.74)'

Say we want to only fit the BMS theorist based on the first input variable. We can do this by changing the state variables, and then assigning the new variable collection to the state. First, we create a new variable collection without the second input variable:

In [ ]:
variables_cut=VariableCollection(
        independent_variables=[Variable("x_1", allowed_values=np.linspace(-10, 10, 10))],
        dependent_variables=[Variable("y")]
    )

Then, we wrap this variable collection into a state (as a dictionary) and then add it to the old state. Here, adding it to the old state will overwrite the old state variables.

In [ ]:
new_state = state+dict(variables = variables_cut)

Let's fit re-fit the BMS theorist.

In [ ]:
new_state = theorist(new_state)
new_state.model.__repr__()


INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 500/500 [00:33<00:00, 14.99it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


'abs((x_1 * 13.8))'